# Lab session 4: data warehousing and online analytical processing

## Introduction 

The purpose of this lab session is to provide you with an opportunity to gain experience with **data warehousing** and **online analytical processing (OLAP)**, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.

This session starts with a tutorial that uses examples to introduce you to the practical knowledge that you will need for the corresponding assignment. We highly recommend that you read the following tutorials if you need a gentler introduction to the libraries that we use:
- [Cubes](https://cubes.readthedocs.io/en/v1.0.1/tutorial.html)
- [Numpy quickstart tutorial](https://numpy.org/devdocs/user/quickstart.html)
- [Numpy: basic broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html)
- [Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html)
- [Matplotlib](https://matplotlib.org/tutorials/introductory/pyplot.html)
- [Seaborn](https://seaborn.pydata.org/tutorial/relational.html)
- [Scikit-learn](https://scikit-learn.org/stable/tutorial/basic/tutorial.html)


## Submission instructions

- The last section of this notebook includes the second part of the assessed assignment for day 2.
- There will be 4 assessed assignments. Each assignment corresponds to 10% of your final grade.
- The exam corresponds to the remaining 60% of your final grade.
- **PLAGIARISM** <ins>leads to irreversible non-negotiable failure in the module</ins> (if you are unsure about what constitutes plagiarism, please ask). 
- The deadline for submitting each assignment is defined on QM+.
- Penalties for late submissions will be applied in accordance with the School policy. The submission cut-off date is 7 days after the deadline.
- You should submit a **report** with your solutions to the assignment included in the last section of this notebook. The report should be a single **pdf** file. Other formats (such as doc, docx, or ipynb) are **not** acceptable. 
- The report should be excellently organized and identified with your name, student number, assignment number, and module identifier. When applicable, question numbers should precede the corresponding answers. 
- Please name your report file according to the following convention: Assignment[Number]-[Student Name]-[Student Number].pdf
- Submissions should be made through QM+. Submissions by e-mail will be ignored.
- Please always check whether the report file was uploaded correctly to QM+.
- Cases of **extenuating circumstances** have to go through the proper procedure in accordance with the School policy. Only cases approved by the School in due time will be considered.

## 1. Introduction to Cubes

This chapter describes how to use ``cubes`` (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for the development of reporting and analytical applications, online analytical processing (OLAP), multidimensional analysis, and browsing of aggregated data. ``cubes`` features:
- a logical view of analysed data
- OLAP and aggregated browsing (default backend is for relational database: ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data 
- authentication and authorization of cubes and their data
- pluggable data warehouse

``cubes`` is meant to be used by application builders that want to provide analytical functionality. ``cubes`` also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

In order to install ``cubes`` in your virtual machine, run the following command:

In [2]:
!pip install cubes

## 2. Data Preparation

This example uses the CSV file ``data/IBRD_Balance_Sheet__FY2010.csv`` (International Bank for Reconstruction and Development Balance Sheet), which should be uploaded to the virtual machine. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in millions of dollars). We first start with imports:

In [4]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table, and populate it with contents of the CSV file:

In [7]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

## 3. Creating a data cube

Everything in ``cubes`` happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes, and more.

The first thing that we have to do is to specify a data store: a database which will host the cube’s data.

In [8]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'data/tutorial_model.json' (as usual, this file should be uploaded to the virtual machine), which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [9]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the JSON file above. This will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table:

In [10]:
cube = workspace.cube("ibrd_balance")

## 4. Aggregations and OLAP operations

A *browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [11]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [12]:
result = browser.aggregate()
result.summary["record_count"]

62

For computing a sum of the amount:

In [13]:
result.summary["amount_sum"]

1116860

Now we can try to compute aggregates by year:

In [14]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'record_count': 31}
{'year': 2010, 'amount_sum': 566020, 'record_count': 31}


Or compute aggregates by item category:

In [15]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 558430, 'record_count': 32}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 77592, 'record_count': 8}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 480838, 'record_count': 22}


We can also perform *slicing* and *dicing* operations on the data cube. The example below performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest: a portion of the cube to be aggregated or browsed.

In [16]:
import cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'amount_sum': 275420, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'amount_sum': 40037, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'amount_sum': 235383, 'record_count': 11}


It's worth noting that slicing operations in ``cubes`` can be created by either specifying a "point cut", which selects a single value of an attribute in a given dimension (using the ``cubes.PointCut`` function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut uses the ``cubes.RangeCut`` function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The example below performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [17]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'amount_sum': 275420, 'record_count': 16}

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [19]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'amount_sum': 123065, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'amount_sum': 3044, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'amount_sum': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'amount_sum': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', 'amount_sum': 1202, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'oa', 'item.subcategory_label': 'Other Assets', 'amount_sum': 2247, 'record_count': 3}
{'ite

# Assignment 2 [Part 2/2]

1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and the two measures count (how many times a user listened to the song) and fee (fee paid by the streaming company to the artist every time a user listens to that song). 
  1. Draw a schema diagram for the above data warehouse using either a star, snowflake, or fact constellation schema. [0.5/5]
  2. Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g., October 2020)? [1/5]


2. Suppose that we have access to a data cube that contains information on rainfall for specific regions. The data cube has dimensions region, precipitation, and time.
  1. Assuming that we would like to compute the total amount of rainfall for a given region and month, which *data cube measure* would we use? To which category of data cube measures does this particular measure fall into? [0.25/5]
  2. Assuming that we would like to compute the average rainfall for a given region and month, which *data cube measure* would we use? To which category of data cube measures does this particular measure fall into? [0.25/5]
    

3. Using the same CSV file and data cube in the tutorial above, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your work in the report. [1.0/5]


4. Using the CSV file ``data/country-income.csv``, perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your report, show the relevant scripts and files. [1.0/5]
  2. Using the created data cube and data cube model, produce aggregate results for the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50. [1/5]
  